In [1]:
from langchain import HuggingFacePipeline
import torch

llm = HuggingFacePipeline.from_model_id(
    model_id="bigscience/bloom-1b7",
    task="text-generation",
    model_kwargs={"temperature": 0, "max_length": 64},
    device=0,
)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
z = llm.generate(['my favorite color is'], 
                 max_new_tokens=5, return_dict_in_generate=True, output_scores=True)
z

In [9]:
from transformers import GPT2Tokenizer, AutoModelForCausalLM

In [13]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer.pad_token_id = tokenizer.eos_token_id
inputs = tokenizer(["Today is"], return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=5, return_dict_in_generate=True, output_scores=True)
transition_scores = model.compute_transition_scores(
    outputs.sequences, outputs.scores, normalize_logits=True
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [15]:
import numpy as np
input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
generated_tokens = outputs.sequences[:, input_length:]
for tok, score in zip(generated_tokens[0], transition_scores[0]):
    # | token | token string | logits | probability
    print(f"| {tok:5d} | {tokenizer.decode(tok):8s} | {score.numpy():.3f} | {np.exp(score.numpy()):.2%}")


|   262 |  the     | -1.414 | 24.33%
|  1110 |  day     | -2.609 | 7.36%
|   618 |  when    | -2.010 | 13.41%
|   356 |  we      | -1.859 | 15.58%
|   460 |  can     | -2.508 | 8.14%


In [ ]:
z.seq

In [8]:
z.generations[0]

[Generation(text=' red. I love red, and I love red lips. I think red is the perfect color for a red lipstick. I think red is the perfect color for a red lipstick. I think red is the perfect color for a red lipstick. I think red is the perfect color for a', generation_info=None)]